In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from util import gloveutil, const

In [3]:
fm = const.FileManager()
glove_200 = gloveutil.load_glove(fm.GLOVE_200)

In [26]:
from util import reader
df = reader.load_df_multi()

In [27]:
maxlen = 30
nwords = 10000
embedding_dim = 200

In [28]:
from sklearn.model_selection import train_test_split
import keras.preprocessing.text as Ktext
import keras.preprocessing.sequence as Kseq

xs = df.body_content
tokenizer = Ktext.Tokenizer(num_words=nwords)
tokenizer.fit_on_texts(xs)
xs_seq = Kseq.pad_sequences(
    tokenizer.texts_to_sequences(xs), maxlen=maxlen)

In [29]:
import numpy as np
model_2_dim = 20

x_train = xs_seq
x_train_2 = np.zeros((xs_seq.shape[0], model_2_dim))
y_train = df.label.values

In [30]:
df.body_atreplies = df.body_atreplies.apply(lambda r: r if isinstance(r, list) else [])

In [31]:
df[[type(r) != list for r in df.body_atreplies]]

,children,created_at,favorite_count,id,label,media,parents,reply_count,retweet_count,user_handle,user_id,user_is_verified,user_name,body_atreplies,body_content,body_hashtags,body_links


In [32]:
user_index = dict()
i = 0
for atreplies in df.body_atreplies:
    for atreply in filter(lambda x: x not in user_index, atreplies):
        user_index[atreply] = i
        i += 1
len(user_index)

8776

In [33]:
users_matrix = np.zeros((len(df), len(user_index)))
for row, v in enumerate(df.body_atreplies.head()):
    for user in v:
        users_matrix[row, user_index[user]] = 1

In [34]:
print(xs.iloc[0])
print(xs_seq[0])
print([k for k, v in tokenizer.word_index.items() if v == 96])
print(users_matrix.shape)

@21WIRE #FatherPed #VicarOfFiddly tune into http://bambuser.com/channel/Joe_Public ... ... from 5pm BST to see #Middlesbrough church shamed Happy Easter
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0 1554 5144  139    2    1 2046 2524  513   29    6  186 2081
 1100  604]
['your']
(46754, 8776)


In [35]:
from keras.layers import Input, Embedding, Layer
from keras.models import Model

embedding_matrix = gloveutil.glove_to_embedding_matrix(
    glove_200, tokenizer.word_index)
X = Input(shape=(maxlen, ))
embed = Embedding(
  len(tokenizer.word_index), embedding_dim, weights=[embedding_matrix],
  input_length=maxlen, trainable=False)

In [36]:
import keras.layers as Klay
import keras.regularizers as Kreg
from keras import Sequential

model_seq = Sequential([
    embed,
    #Klay.Conv1D(filters=100, kernel_size=5, padding='same'),
    #Klay.MaxPool1D(pool_size=2),
    #Klay.Conv1D(filters=100, kernel_size=5, padding='same'),
    Klay.LSTM(100),
    #Klay.Flatten(),
    Klay.Dense(100, activation='relu'),
])

model_seq2 = Sequential([
    Klay.Dense(100, input_shape=(users_matrix.shape[1],)),
    Klay.Dense(100)
])

concat = Klay.concatenate([model_seq.output, model_seq2.output])
Y = Klay.Dense(4, activation='sigmoid')(concat)
model_combined = Model([model_seq.input, model_seq2.input], Y)

In [61]:
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

class Metrics(Callback):
    
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_f1macros = []
 
    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(
            [self.validation_data[0], self.validation_data[1]]))).round()
        val_targ = self.validation_data[2]
        
        _val_f1 = f1_score(val_targ, val_predict, average='micro')
        _val_f1_macro = f1_score(val_targ, val_predict, average='macro')
        
        self.val_f1s.append(_val_f1)
        self.val_f1macros.append(_val_f1_macro)
        
        print("— f1: %f — f1_macro: %f" % (_val_f1, _val_f1_macro))

metrics = Metrics()

In [62]:
from keras.utils import to_categorical

In [63]:
model_combined.compile(
    loss='categorical_crossentropy',
    optimizer='adam',)
model_combined.fit(
    [x_train, users_matrix], 
    to_categorical(y_train),
    batch_size=128,
    epochs=10,
    validation_split=0.2,
    callbacks=[metrics])

Train on 37403 samples, validate on 9351 samples
Epoch 1/10
37403/37403 [==============================] - 36s 972us/step - loss: 0.1227 - val_loss: 0.1991
— f1: 0.276586 — f1_macro: 0.283090
Epoch 2/10
37403/37403 [==============================] - 35s 928us/step - loss: 0.1057 - val_loss: 0.2244
— f1: 0.473432 — f1_macro: 0.340725
Epoch 3/10
  128/37403 [..............................] - ETA: 34s - loss: 0.1032

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


37403/37403 [==============================] - 34s 908us/step - loss: 0.0926 - val_loss: 0.2929
— f1: 0.416653 — f1_macro: 0.327935
Epoch 4/10
37403/37403 [==============================] - 34s 911us/step - loss: 0.0775 - val_loss: 0.2597
— f1: 0.387441 — f1_macro: 0.315351
Epoch 5/10
37403/37403 [==============================] - 37s 1ms/step - loss: 0.0664 - val_loss: 0.2143
— f1: 0.550541 — f1_macro: 0.366437
Epoch 6/10
37403/37403 [==============================] - 35s 929us/step - loss: 0.0532 - val_loss: 0.3060
— f1: 0.641774 — f1_macro: 0.390703
Epoch 7/10
37403/37403 [==============================] - 36s 962us/step - loss: 0.0474 - val_loss: 0.3132
— f1: 0.568427 — f1_macro: 0.370888
Epoch 8/10
37403/37403 [==============================] - 34s 918us/step - loss: 0.0335 - val_loss: 0.3324
— f1: 0.669633 — f1_macro: 0.398045
Epoch 9/10
37403/37403 [==============================] - 39s 1ms/step - loss: 0.0300 - val_loss: 0.4177
— f1: 0.683569 — f1_macro: 0.404932
Epoch 10/10
37